# Baseline Experiments

In [9]:
# %pip install langchain langchain-core langchain-community langchain-openai
# %pip install --force-reinstall typing-extensions==4.5
# %pip install --force-reinstall openai==1.8

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached typing_extensions-4.10.0-py3-none-any.whl.metadata (3.0 kB)
Using cached typing_extensions-4.10.0-py3-none-any.whl (33 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are us

## Load Libraries

In [1]:
import sys
import pandas as pd
sys.path.append('..')
from src.prompt import PROMPTS
from src.evaluation import Evalator
from src.helpers import fix_decision_parser
from src.experiment_runner import run_experiment

from huggingface_hub import notebook_login
from datasets import Dataset, DatasetDict

pd.set_option('display.max_colwidth', None)
notebook_login()

/opt/homebrew/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


## Load Data

In [2]:
df_uk = pd.read_csv('../data/uk_data_samples.csv')
df_en = pd.read_csv('../data/en_data_samples.csv')

## `gpt-3.5-turbo-0125`: English Language experiment

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", model_kwargs={"seed": 42})

In [4]:
chain = PROMPTS['baseline_prompt_en'] | llm

In [5]:
file_paths = run_experiment(
    folder_path='../data/baseline',
    chain=chain,
    data=df_en,
    lang='en',
    batch_size=50,
)

In [6]:
file_paths

{}

## Load English Results to HF Datasets

In [7]:
FILES_PATHS = {
    'gender': '../data/baseline/en/gender.csv',
    'marital_status': '../data/baseline/en/marital_status.csv',
    'military_status': '../data/baseline/en/military_status.csv',
    'religion': '../data/baseline/en/religion.csv',
    'name': '../data/baseline/en/name.csv',
    'age': '../data/baseline/en/age.csv'
    }

# load data and push to huggingface
dataset_dict = {}
for key_name, file_path in FILES_PATHS.items():
    df_part = pd.read_csv(file_path, dtype={'protected_attr': str})
    df_part['decision'] = df_part['decision'].map(lambda x: x.lower())
    df_part = df_part.groupby(by=['candidate_id', 'job_id','CV','Job Description', 'Job Position', 'lang', 'protected_group', 'protected_attr', 'group_id']).agg({
        "decision": "first",
        "feedback": "first",
        "raw_ai_decision": "first",
    }).reset_index()
    dataset_dict[key_name] = Dataset.from_pandas(df_part)

DatasetDict(dataset_dict).push_to_hub('Stereotypes-in-LLMs/hiring-analyses-baseline-en', private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Stereotypes-in-LLMs/hiring-analyses-baseline-en/commit/69812189232fac7236bb6a6e360480a0ecb3c4a6', commit_message='Upload dataset', commit_description='', oid='69812189232fac7236bb6a6e360480a0ecb3c4a6', pr_url=None, pr_revision=None, pr_num=None)

In [8]:
# # load data from huggingface
# from datasets import load_dataset
# dataset = load_dataset("Stereotypes-in-LLMs/hiring-analyses-baseline-en", split="age")
# dataset

## Evaluation Scores

In [9]:
evaluator = Evalator("intfloat/multilingual-e5-large", "Stereotypes-in-LLMs/hiring-analyses-baseline-en", "baseline")

/opt/homebrew/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Generating gender split:   0%|          | 0/9000 [00:00<?, ? examples/s]

Generating marital_status split:   0%|          | 0/2250 [00:00<?, ? examples/s]

Generating military_status split:   0%|          | 0/2250 [00:00<?, ? examples/s]

Generating religion split:   0%|          | 0/4050 [00:00<?, ? examples/s]

Generating name split:   0%|          | 0/4500 [00:00<?, ? examples/s]

Generating age split:   0%|          | 0/2700 [00:00<?, ? examples/s]

In [10]:
df_report = evaluator.get_report()
df_report

,experiment_name,protected_group,lang,min_feedback_similarity,median_feedback_similarity,max_feedback_similarity,mean_reject_approve_per_attr,mean_bias_per_attr
0,baseline,gender,en,0.7957,0.9480,1.0,"{'Agender': 0.6, 'Androgynous': 0.6089, 'Bigender': 0.5289, 'Cisgender': 0.6, 'Demiboy': 0.5867, 'Demigender': 0.5156, 'Demigirl': 0.5911, 'Female': 0.5956, 'Gender Nonconforming': 0.5778, 'Genderfluid': 0.5622, 'Genderqueer': 0.5756, 'Intersex': 0.5356, 'Male': 0.5333, 'Neutrois': 0.58, 'Non-Binary': 0.6356, 'Pangender': 0.5022, 'Queer': 0.5556, 'Third Gender': 0.5111, 'Transgender': 0.5756, 'Two-Spirit': 0.5267}","{'Agender': 0.08, 'Androgynous': 0.0756, 'Bigender': 0.0489, 'Cisgender': 0.0711, 'Demiboy': 0.0533, 'Demigender': 0.0667, 'Demigirl': 0.0711, 'Female': 0.0622, 'Gender Nonconforming': 0.08, 'Genderfluid': 0.0556, 'Genderqueer': 0.0689, 'Intersex': 0.0467, 'Male': 0.0711, 'Neutrois': 0.0733, 'Non-Binary': 0.0889, 'Pangender': 0.0756, 'Queer': 0.0356, 'Third Gender': 0.0711, 'Transgender': 0.0733, 'Two-Spirit': 0.0733}"
1,baseline,marital_status,en,0.8160,0.9466,1.0,"{'Civil union': 0.5267, 'Divorced (Divorced)': 0.5578, 'Married (Husband/Wife)': 0.5467, 'Unmarried (Single)': 0.5644, 'Widower (Widow)': 0.5267}","{'Civil union': 0.0756, 'Divorced (Divorced)': 0.0533, 'Married (Husband/Wife)': 0.0467, 'Unmarried (Single)': 0.0778, 'Widower (Widow)': 0.0533}"
2,baseline,military_status,en,0.7607,0.9380,1.0,"{'Civilian': 0.5889, 'Military retiree': 0.52, 'Participant in combat actions': 0.2978, 'Reservist': 0.54, 'War veteran': 0.5178}","{'Civilian': 0.0889, 'Military retiree': 0.06, 'Participant in combat actions': 0.2156, 'Reservist': 0.0533, 'War veteran': 0.0756}"
3,baseline,religion,en,0.7758,0.9491,1.0,"{'atheist': 0.5911, 'buddhist': 0.4356, 'christian': 0.56, 'hindu': 0.4778, 'jain': 0.48, 'jew': 0.4911, 'muslim': 0.4911, 'sikh': 0.4689, 'zoroastrian': 0.44}","{'atheist': 0.0956, 'buddhist': 0.0778, 'christian': 0.0644, 'hindu': 0.0578, 'jain': 0.0689, 'jew': 0.0667, 'muslim': 0.0622, 'sikh': 0.0578, 'zoroastrian': 0.1044}"
4,baseline,name,en,0.7456,0.8980,1.0,"{'Aaron': 0.5778, 'Amartol': 0.5533, 'Emma': 0.5844, 'Khulian': 0.5511, 'Liusia': 0.5689, 'Liusiia': 0.5756, 'Romchyk': 0.5533, 'Siu': 0.5422, 'Tyhran': 0.5711, 'Zenoviia': 0.5778}","{'Aaron': 0.0533, 'Amartol': 0.0467, 'Emma': 0.0467, 'Khulian': 0.04, 'Liusia': 0.0311, 'Liusiia': 0.06, 'Romchyk': 0.0467, 'Siu': 0.0622, 'Tyhran': 0.0422, 'Zenoviia': 0.0444}"
5,baseline,age,en,0.7630,0.9346,1.0,"{'20': 0.4467, '30': 0.5556, '40': 0.4267, '50': 0.2667, '60': 0.1444, '70': 0.06}","{'20': 0.1867, '30': 0.2911, '40': 0.1622, '50': 0.0067, '60': 0.1244, '70': 0.2044}"


In [11]:
EVALUATION_FILE  = '../data/evaluation_results_en.csv'
evaluator.save_report(df_report, EVALUATION_FILE)

Report saved to ../data/evaluation_results_en.csv


## `gpt-3.5-turbo-0125`: Ukrainian Language experiment

In [5]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", model_kwargs={"seed": 42})

In [6]:
chain = PROMPTS['baseline_prompt_uk'] | llm

In [7]:
file_paths = run_experiment(
    folder_path='../data/baseline',
    chain=chain,
    data=df_uk,
    lang='uk',
    batch_size=25,
)

In [8]:
file_paths

{'gender': '../data/baseline/uk/gender.csv'}

## Load Ukrainian Results to HF Datasets

In [12]:
FILES_PATHS = {
    'gender': '../data/baseline/uk/gender.csv',
    'marital_status': '../data/baseline/uk/marital_status.csv',
    'military_status': '../data/baseline/uk/military_status.csv',
    'religion': '../data/baseline/uk/religion.csv',
    'name': '../data/baseline/uk/name.csv',
    'age': '../data/baseline/uk/age.csv'
    }

def ukr_decision_fixing(x: str) -> str:
    if not x:
        return x
    for word in x.split():
        for part in ['найн', 'наїн', 'наєн', 'прий', 'accept', 'hire']:
            if word.lower().startswith(part):
                return 'hire'
        for part in ['відхил', 'reject', 'відмов', 'вибачте']:
            if word.lower().startswith(part):
                return 'reject'
    return x


# load data and push to huggingface
dataset_dict = {}
for key_name, file_path in FILES_PATHS.items():
    df_part = pd.read_csv(file_path, dtype={'protected_attr': str})

    df_part = fix_decision_parser(df_part)
    df_part['decision'] = df_part['decision'].map(ukr_decision_fixing)
    df_part = df_part.groupby(by=['candidate_id', 'job_id','CV','Job Description', 'Job Position', 'lang', 'protected_group', 'protected_attr', 'group_id']).agg({
        "decision": "first",
        "feedback": "first",
        "raw_ai_decision": "first",
    }).reset_index()
    dataset_dict[key_name] = Dataset.from_pandas(df_part)

DatasetDict(dataset_dict).push_to_hub('Stereotypes-in-LLMs/hiring-analyses-baseline-uk', private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Stereotypes-in-LLMs/hiring-analyses-baseline-uk/commit/c66de6c3ac96b0b488690e335227893ce40f6532', commit_message='Upload dataset', commit_description='', oid='c66de6c3ac96b0b488690e335227893ce40f6532', pr_url=None, pr_revision=None, pr_num=None)

In [13]:
# # load data from huggingface
# from datasets import load_dataset
# dataset = load_dataset("Stereotypes-in-LLMs/hiring-analyses-baseline-uk", split="gender")
# dataset.to_pandas().head()

## Evaluation Scores

In [14]:
evaluator = Evalator("intfloat/multilingual-e5-large", "Stereotypes-in-LLMs/hiring-analyses-baseline-uk", "baseline")

Generating gender split:   0%|          | 0/9000 [00:00<?, ? examples/s]

Generating marital_status split:   0%|          | 0/2250 [00:00<?, ? examples/s]

Generating military_status split:   0%|          | 0/2250 [00:00<?, ? examples/s]

Generating religion split:   0%|          | 0/4050 [00:00<?, ? examples/s]

Generating name split:   0%|          | 0/4500 [00:00<?, ? examples/s]

Generating age split:   0%|          | 0/2700 [00:00<?, ? examples/s]

In [15]:
df_report = evaluator.get_report()
df_report

,experiment_name,protected_group,lang,min_feedback_similarity,median_feedback_similarity,max_feedback_similarity,mean_reject_approve_per_attr,mean_bias_per_attr
0,baseline,gender,uk,0.7906,0.9389,1.0000,"{'Інтерсекс': 0.1622, 'Агендер': 0.14, 'Андрогінний': 0.1933, 'Бігендер': 0.18, 'Гендерквір': 0.1667, 'Гендерне невідповідність': 0.0622, 'Гендерфлюїд': 0.1311, 'Дводушний (Твоуспірит)': 0.2356, 'Демігендер': 0.1533, 'Демідівчина': 0.2156, 'Деміхлопчик': 0.12, 'Квір': 0.1489, 'Небінарний': 0.2289, 'Неутроїс': 0.1756, 'Пангендер': 0.1889, 'Трансгендер': 0.0822, 'Третя стать': 0.1267, 'Цісгендер': 0.1756, 'Чоловік': 0.1756, 'Жінка': 0.1756}","{'Інтерсекс': 0.1044, 'Агендер': 0.0822, 'Андрогінний': 0.1222, 'Бігендер': 0.1, 'Гендерквір': 0.0956, 'Гендерне невідповідність': 0.0756, 'Гендерфлюїд': 0.0911, 'Дводушний (Твоуспірит)': 0.1689, 'Демігендер': 0.1044, 'Демідівчина': 0.14, 'Деміхлопчик': 0.0667, 'Квір': 0.0911, 'Небінарний': 0.1533, 'Неутроїс': 0.1, 'Пангендер': 0.1133, 'Трансгендер': 0.0822, 'Третя стать': 0.0689, 'Цісгендер': 0.0911, 'Чоловік': 0.1133, 'Жінка': 0.1133}"
1,baseline,marital_status,uk,0.8416,0.9401,1.0000,"{'Вдовець/Вдова': 0.14, 'Неодружений/Неодружена': 0.2267, 'Одружений/Одружена': 0.2511, 'Розлучений/Розлучена': 0.1356, 'Цивільний шлюб': 0.2244}","{'Вдовець/Вдова': 0.0644, 'Неодружений/Неодружена': 0.1467, 'Одружений/Одружена': 0.1044, 'Розлучений/Розлучена': 0.0467, 'Цивільний шлюб': 0.1489}"
2,baseline,military_status,uk,0.8116,0.9368,0.9998,"{'Ветеран війни': 0.0956, 'Військовий пенсіонер': 0.0556, 'Резервіст': 0.0911, 'Учасник бойових дій': 0.0533, 'Цивільний': 0.1178}","{'Ветеран війни': 0.0489, 'Військовий пенсіонер': 0.0311, 'Резервіст': 0.0489, 'Учасник бойових дій': 0.0422, 'Цивільний': 0.0578}"
3,baseline,religion,uk,0.7810,0.9394,1.0000,"{'атеїст': 0.16, 'буддист': 0.0867, 'джайніст': 0.0689, 'зороастрист': 0.1022, 'мусульманин': 0.0444, 'сикх': 0.1222, 'християнин': 0.14, 'єврей': 0.1244, 'індуїст': 0.0933}","{'атеїст': 0.1133, 'буддист': 0.0489, 'джайніст': 0.0489, 'зороастрист': 0.0511, 'мусульманин': 0.0289, 'сикх': 0.0711, 'християнин': 0.1111, 'єврей': 0.0733, 'індуїст': 0.0422}"
4,baseline,name,uk,0.8144,0.9323,1.0000,"{'Аарон': 0.2133, 'Амартол': 0.2333, 'Емма': 0.2667, 'Зеновія': 0.2489, 'Люся': 0.2622, 'Люсія': 0.28, 'Ромчик': 0.1644, 'Сю': 0.2289, 'Тигран': 0.2533, 'Хуліан': 0.2422}","{'Аарон': 0.0822, 'Амартол': 0.1156, 'Емма': 0.1178, 'Зеновія': 0.1222, 'Люся': 0.1222, 'Люсія': 0.1178, 'Ромчик': 0.1178, 'Сю': 0.1289, 'Тигран': 0.1267, 'Хуліан': 0.1333}"
5,baseline,age,uk,0.8153,0.9423,1.0000,"{'20': 0.1533, '30': 0.1778, '40': 0.1, '50': 0.0489, '60': 0.02, '70': 0.02}","{'20': 0.1067, '30': 0.1311, '40': 0.0533, '50': 0.0111, '60': 0.0311, '70': 0.0311}"


In [16]:
EVALUATION_FILE  = '../data/evaluation_results_uk.csv'
evaluator.save_report(df_report, EVALUATION_FILE)

Report saved to ../data/evaluation_results_uk.csv


: 